# **Face Blurring for Privacy:**
## *A Video Anonymization Tool*

**Note to Users:**

For optimal performance and faster rendering times, it is recommended to use this tool with Google Colab. You can easily run the provided Colab notebook by following these steps:

1. Open Google Colab: [Google Colab](https://colab.research.google.com/)

2. Create a New Notebook.

3. Copy and Paste the provided code into the notebook.

4. Mount your Google Drive using the code cell provided.

5. Upload your video file to your Google Drive and adjust the file paths in the script accordingly.

6. Run the cells in the notebook to execute the face-blurring script.

By using Google Colab, you'll benefit from the cloud-based computation, significantly reducing render times.

Mounting Google Drive in Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Mediapipe Library

In [ ]:
!pip install Mediapipe

In [ ]:
import cv2
import mediapipe as mp
from moviepy.editor import VideoFileClip
import os
import numpy as np

# Load the video from your drive
video_path = "/content/drive/MyDrive/PrivacyGuard/sample.mp4"  # <----- Input your file here
final_output_path = '/content/drive/MyDrive/PrivacyGuard/result.mp4'  # <----- Output file

# Get the frames per second (fps) of the input video
video_capture = cv2.VideoCapture(video_path)
fps = video_capture.get(cv2.CAP_PROP_FPS)

# Get video width and height
width = int(video_capture.get(3))
height = int(video_capture.get(4))

# Initialize a list to store processed frames
processed_frames = []

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

while True:
    # Read each frame from the video
    ret, frame = video_capture.read()
    if not ret:
        break

    # Resize the frame to reduce processing time
    frame = cv2.resize(frame, (width, height))

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe FaceMesh
    result = face_mesh.process(rgb_frame)

    # If faces are detected, apply stronger mosaic blur
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            landmarks = np.array([(landmark.x * width, landmark.y * height) for landmark in face_landmarks.landmark])

            # Convert the landmarks to a list of tuples
            landmarks_list = [(int(landmark[0]), int(landmark[1])) for landmark in landmarks]

            # Convert landmarks_list to NumPy array
            landmarks_np = np.array(landmarks_list)

            # Expand the region around the face landmarks to capture the entire head
            x, y, w, h = cv2.boundingRect(landmarks_np)
            expanded_x = max(0, x - int(0.2 * w))
            expanded_y = max(0, y - int(0.2 * h))
            expanded_w = min(width, x + w + int(0.2 * w)) - expanded_x
            expanded_h = min(height, y + h + int(0.2 * h)) - expanded_y

            # Extract the expanded face region
            sub_face = frame[expanded_y:expanded_y + expanded_h, expanded_x:expanded_x + expanded_w]

            # Apply stronger mosaic blur directly
            sub_face = cv2.resize(sub_face, (10, 10),
                                  interpolation=cv2.INTER_NEAREST)
            sub_face = cv2.resize(sub_face, (expanded_w, expanded_h),
                                  interpolation=cv2.INTER_NEAREST)
            frame[expanded_y:expanded_y + expanded_h, expanded_x:expanded_x + expanded_w] = sub_face

    # Append the processed frame to the list
    processed_frames.append(frame)

# Release the video capture object
video_capture.release()

# Write the processed frames to a new video file
output_path = '/content/drive/MyDrive/Project/temp_without_audio.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

for frame in processed_frames:
    out.write(frame)

# Release the video writer object
out.release()

original_clip = VideoFileClip(video_path)
processed_clip = VideoFileClip(output_path)
final_clip = processed_clip.set_audio(original_clip.audio)

# Write the final video with audio
final_clip.write_videofile(final_output_path, codec='libx264', audio_codec='aac',
                           fps=original_clip.fps, preset="ultrafast")

# Close the video clips
original_clip.close()
processed_clip.close()

# Delete the temp data files
os.remove(output_path)

Moviepy - Building video /content/drive/MyDrive/PrivacyGuard/result.mp4.
MoviePy - Writing audio in resultTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PrivacyGuard/result.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PrivacyGuard/result.mp4
